In [ ]:
from Dataset import DatasetBuilder

start_directory = "C:\\Users\\gciprianherrera\\Desktop\\LLM\\MVP_Chatbot\\PDF"
pdf_processor = DatasetBuilder(start_directory)
data = pdf_processor.process_directory()

In [2]:

import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings
model: str = "text-embedding-ada-002"


from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model=os.getenv("AZURE_OPENAI_MODEL"),
    temperature=0,
)

In [3]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_EMB_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_EMB_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_EMB_VERSION"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMB_DEPLOYMENT_NAME"),
)

In [6]:
index_name: str = "complete_dataset"
vector_store: AzureSearch = AzureSearch(
    azure_search_key = os.getenv("AZURE_AI_SEARCH_API_KEY"),
    azure_search_endpoint = os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    index_name=index_name,
    embedding_function=embeddings.embed_query
)

In [7]:
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(data, page_content_column="page_content")
documents = loader.load()

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=5,
    length_function=tiktoken_len,
    separators=["\n", " ", ""])
splits = text_splitter.split_documents(documents)
vector_store.add_documents(documents=splits)

In [9]:
AZURE_AI_SEARCH_API_KEY=os.getenv("AZURE_AI_SEARCH_API_KEY")
AZURE_AI_SEARCH_ENDPOINT=os.getenv("AZURE_AI_SEARCH_ENDPOINT")
AZURE_AI_SEARCH_DEPLOYEMENT_ID=os.getenv("AZURE_AI_SEARCH_DEPLOYEMENT_ID")
AZURE_AI_SEARCH_INDEX_NAME=os.getenv("AZURE_AI_SEARCH_INDEX_NAME")
AZURE_AI_SEARCH_SERVICE_NAME=os.getenv("AZURE_AI_SEARCH_SERVICE_NAME")

In [3]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import AzureChatOpenAI

from langchain_community.retrievers import (
    AzureAISearchRetriever,   
)

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model="gpt-35-turbo",
    temperature=0,
)

index_name: str = "complete_dataset"
retriever = AzureAISearchRetriever(
    content_key="content", top_k=30, index_name=index_name
)

# query = "Quel est l'objectif du compte-rendu de la cellule 951 DEG"  
# query = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904"  
# query = "Quels sont les équipements investigués sur le périmètre DEG"  
# query = "Quel est le débit de dose de la 951 du DEG " 
# query = "Quelles sont les informations de la cellule 909 du DEG " 
# query = "Quelle est la quantité de boue de la fosse 217-02 du DEG"  
# query = "DDD des PE de boues sur la fosse 217-02 du DEG " 
# query = "Quels sont les enseignements sur les investigations de la cellule 959B du HADE"
query = "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE"

system_prompt = (
    "Use the given context to answer the question. "
    "If you don't know the answer, say you don't know. "
    "Use five sentence maximum and keep the answer concise. "
    "Context: {context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(retriever, question_answer_chain)
chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | AzureAISearchRetriever(service_name='fcoaysearchopenai01', index_name='complete_dataset', api_key='7Ib46eoW4qPUTahv3UmyBVT8ua5oBTUQyvdfZSuhabAzSeAUgeBA', top_k=30), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the given context to answer the question. If you don't know the answer, say you don't know. Use five sentence maximum and keep the answer concise. Context: {context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
      

In [7]:
chain.invoke({"input": query})

{'input': "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE",
 'context': [Document(page_content='Figure 2 : Plan d’ensemble du platelage en 850 – Extrait du plan [3] \nCaractéristiques de la cellule 929A \nSes dimensions sont les suivantes : \n• \nLargeur → 3,15m ; \n• \nLongueur → 3,62m ; \n• \nHauteur → 2,80m.', metadata={'@search.score': 19.480326, 'id': 'ZWI5YWRjYjYtZTdhNC00YTgwLTg3ZjYtZjgyYzk2MDM3MTRk', 'content_vector': [-0.0025177759, 0.008815486, 0.012726213, 0.0018719827, -0.035575848, 0.014151863, -0.008658533, -0.0033973625, -0.01338672, 0.009011676, 0.031756677, 0.01156869, -0.0029085216, -0.009606787, -0.005352726, 0.01867732, 0.012412309, 0.00068625744, -0.00446006, -0.021790206, 0.0031325056, 0.0024311251, -0.008383866, 0.0019962366, 0.0066900905, 0.006536408, 0.006958217, -0.021332428, 0.011006278, -0.0034660292, 0.012830848, 0.003060569, -1.0106485e-06, -0.010273834, -0.0026011567, 0.0032567596, 0.018651161, 0.021384746, 0.040886067, 0.015

In [16]:
type(chain.invoke({"input": query})["context"][0])

langchain_core.documents.base.Document

In [5]:
class Chatbot:
    def __init__(self):
        self.llm = AzureChatOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version="2023-05-15",
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            model="gpt-35-turbo",
            temperature=0,
        )

        self.index_name = "complete_dataset"
        self.retriever = AzureAISearchRetriever(
            content_key="content", top_k=30, index_name=self.index_name
        )

        system_prompt = (
            "Use the given context to answer the question. "
            "If you don't know the answer, say you don't know. "
            "Use five sentences maximum and keep the answer concise. "
            "Context: {context}"
        )
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", "{input}"),
            ]
        )
        self.question_answer_chain = create_stuff_documents_chain(self.llm, prompt)
        self.chain = create_retrieval_chain(self.retriever, self.question_answer_chain)

    def query(self, user_query):
        response = self.chain.invoke({"input": query})['answer']
        return response


# query = "Quel est l'objectif du compte-rendu de la cellule 951 DEG"  
# query = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904"  
# query = "Quels sont les équipements investigués sur le périmètre DEG"  
# query = "Quel est le débit de dose de la 951 du DEG " 
# query = "Quelles sont les informations de la cellule 909 du DEG " 
# query = "Quelle est la quantité de boue de la fosse 217-02 du DEG"  
# query = "DDD des PE de boues sur la fosse 217-02 du DEG " 
# query = "Quels sont les enseignements sur les investigations de la cellule 959B du HADE"
# query = "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE"

In [4]:

from chat import Chatbot

chatbot = Chatbot()
# query = "Quel est l'objectif du compte-rendu de la cellule 951 DEG"  
# query = "Quel est l'objectif de l'investigation sur l’ensemble porte-lame cellule 904"  
# query = "Quels sont les équipements investigués sur le périmètre DEG"  
# query = "Quel est le débit de dose de la 951 du DEG " 
# query = "Quelles sont les informations de la cellule 909 du DEG " 
# query = "Quelle est la quantité de boue de la fosse 217-02 du DEG"  
# query = "DDD des PE de boues sur la fosse 217-02 du DEG " 
query = "Quels sont les enseignements sur les investigations de la cellule 959B du HADE"
# query = "Quelles sont les informations sur l'état de la cellule 929A du périmètre HADE"
response = chatbot.ask(query)
print(response)

Le compte-rendu présente les résultats des investigations dans le pot florentin 239-74 et dans la cuve 239-60 de la cellule 959B du bâtiment HADE. Les objectifs de ces investigations étaient de prélever un échantillon de dépôt présent au fond du pot 239-74 et de définir un profil radiologique du pot, ainsi qu'un pigeage du liquide résiduel présent dans la cuve 239-60 et une vidéo à 360°. Les opérations d'investigation ont été réalisées en février 2021 en utilisant les éléments suivants : N°DIMR : 428107, DT : 8910710, N° Spectre α : 39, N° Spectre βγ : 34. Les vidéos de l'intérieur du pot florentin et de la cuve 239-60 sont disponibles. Les opérations réalisées sont classées par ordre chronologique et ont été les suivantes : dépose des BBL, mise en place porte d'accès cellule 959B HADE et trépanation de la cuve 239-60. Le compte-rendu est complémentaire des autres compte-rendus d'investigation référencés.
